In [4]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import pickle as pkl
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression 
from sklearn.model_selection import train_test_split
import joblib

%run ../dnam-clocks/mFSS-clock/scripts/mfss_model_train.py
%run ../dnam-clocks/mFSS-clock/scripts/utils/data_processing.py
%matplotlib inline

In [2]:
data_path = '../dnam-clocks/mFSS-clock/data/processed/'

In [ ]:
# Read in the Hannum dataset for training
hannum = pd.read_pickle(data_path + 'GSE40279_hannum/hannum32.pkl')
hannum_meta = pd.read_excel('GSE40279_hannum/hannum_pmeta.xlsx')

In [5]:
# Read in the Lehne datasat for validation
lehne = np.load(data_path + 'GSE55763_lehne/lehne32_reduced.npy')
lehne_meta = pd.read_excel(data_path + 'GSE55763_lehne/lehne_pmeta.xlsx')
lehne_cgs = list_reader(data_path + 'GSE55763_lehne/cgs')
lehne = pd.DataFrame(lehne, columns=lehne_cgs)

In [7]:
#get the age_correlations so we can focus on the shifts of just the top N age-correlated CpGs
age_corrs=pd.read_excel(data_path + 'cpg_sets/hannum_age_corrs.xlsx')

In [9]:
#filter out SNPS
manifest = pd.read_csv(data_path + 'metadata/450K_manifest.csv', low_memory=False)
# Get all SNP-associated CpGs in 450K array 
snps = set(manifest[~manifest.Probe_SNPs.isna()].Name)
no_snps = list(set(age_corrs.CpG)-snps)
age_corrs = age_corrs[age_corrs['CpG'].isin(no_snps)]

In [ ]:
#Lehne has some missing CpGs so need to remove those from the age_corrs rows
overlap = list(set(lehne.columns)&set(age_corrs.CpG))

filtered_age_corrs= age_corrs[age_corrs['CpG'].isin(overlap)]
filtered_age_corrs.reset_index(inplace=True)

In [ ]:
model_cgs, best_iter, val_mse, val_r_val, test_mse, test_r_val = mf_stepwise(filtered_age_corrs.CpG.tolist()[:10000], hannum, hannum_meta,
                                                                             lehne, lehne_meta, 150,flag=False)